In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import shutil
import time

In [2]:
filename = "iris.data.csv"

In [3]:
with open(filename) as f:
        for i, l in enumerate(f):
            pass
        data_len = i

In [4]:
def datasplitter(filename):
    train_rows = 0
    valid_rows = 0
    test_rows = 0
    
    if os.path.exists('iristrain'):
        shutil.rmtree('iristrain')
    os.makedirs('iristrain')
    
    
    if os.path.exists('irisvalid'):
        shutil.rmtree('irisvalid')
    os.makedirs('irisvalid')
    
    if os.path.exists('iristest'):
        shutil.rmtree('iristest')
    os.makedirs('iristest')

    with open(filename, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row)>0:
                filtered = (line.replace('\n', '') for line in row)
                filtered = (line.replace("\r", "") for line in filtered)
                filtered = (line.replace("'", "") for line in filtered)
                row = filtered

                group = np.random.multinomial(1,[.6,.1,.3])
                if np.argmax(group)==0:
                    with open('iristrain/iristrain{0}.csv'.format(train_rows), 'w+', newline = None) as trainfile:
                        trainwriter = csv.writer(trainfile, delimiter=',')
                        trainwriter.writerow(row)
                    train_rows += 1


                if np.argmax(group)==1:
                    with open('irisvalid/irisvalid{0}.csv'.format(valid_rows), 'w+', newline = None) as validfile:
                        validwriter = csv.writer(validfile, delimiter=',')
                        validwriter.writerow(row)
                    valid_rows += 1

                if np.argmax(group)==2:
                    with open('iristest/iristest{0}.csv'.format(test_rows), 'w+', newline = None) as testfile:
                        testwriter = csv.writer(testfile, delimiter=',')
                        testwriter.writerow(row)
                    test_rows += 1
                    
    return train_rows, valid_rows, test_rows

In [5]:
train_len, valid_len, test_len = datasplitter(filename=filename)

In [6]:
graph = tf.Graph()
init = tf.global_variables_initializer()

In [7]:
def read_file_format(filename_queue, possible_labels):
    reader = tf.TextLineReader()
    _, value = reader.read(filename_queue)

    record_defaults = [tf.constant([],dtype=tf.float32), tf.constant([],dtype=tf.float32),
                       tf.constant([],dtype=tf.float32), tf.constant([],dtype=tf.float32),
                       tf.constant([],dtype=tf.float32), tf.constant([],dtype=tf.string)]
    col1, col2, col3, col4, col5, col6 = tf.decode_csv(value, record_defaults=record_defaults)
    
    example = tf.stack([col1, col2, col3, col4, col5])
    label = tf.one_hot(tf.where(tf.equal(possible_labels, col6))[0], depth = possible_labels.shape[0], on_value = 1, off_value = 0)
    label = label[0]
    return example, label

In [8]:
def input_pipeline(filenames, possible_labels, batch_size = 3, num_epochs = None, evaluation = False):   
    filename_queue = tf.train.string_input_producer(
        train_set, num_epochs=num_epochs, shuffle=True)
    
    example, label = read_file_format(filename_queue, possible_labels)

#     # min_after_dequeue defines how big a buffer we will randomly sample
#     #   from -- bigger means better shuffling but slower start up and more
#     #   memory used.
#     # capacity must be larger than min_after_dequeue and the amount larger
#     #   determines the maximum we will prefetch.  Recommendation:
#     #   min_after_dequeue + (num_threads + a small safety margin) * batch_size
    min_after_dequeue = 10
    capacity = min_after_dequeue + 3 * batch_size
    example_batch, label_batch = tf.train.shuffle_batch(
      [example, label], batch_size=batch_size,capacity = capacity, 
        min_after_dequeue = min_after_dequeue
    )
    return example_batch, label_batch

In [10]:
num_epochs = 1
batch_size = 3

In [11]:
train_set = ['iristrain/iristrain{0}.csv'.format(i) for i in range(train_len)]
valid_set = ['irisvalid/irisvalid{0}.csv'.format(i) for i in range(valid_len)]


In [12]:
for i in range(train_len):
    if i == 0:
        debug = pd.read_csv('iristrain/iristrain{0}.csv'.format(i), header = None)
    else:
        temp = pd.read_csv('iristrain/iristrain{0}.csv'.format(i), header = None)
        debug = debug.append(temp)
debug_ex = debug.ix[:,:4]
    

In [13]:
pipe_ex = debug_ex.iloc[0:0,:]

In [14]:
with graph.as_default():
    
    unique_labels = tf.constant(["Iris-setosa", "Iris-versicolor", "Iris-virginica"], dtype=tf.string)
    
    example_feed, labels_feed = input_pipeline(train_set, possible_labels = unique_labels,
                                               batch_size = batch_size, num_epochs = num_epochs)
    
    with tf.Session() as sess:
        # Start populating the filename queue.
        
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)
        coord = tf.train.Coordinator()  
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        
        step = 0
        while not coord.should_stop():
            try:
                start_time = time.time()                
               
                example_batch, label_batch = sess.run([example_feed, labels_feed])
                
                pipe_ex = pipe_ex.append(pd.DataFrame(example_batch))
                
                duration = time.time() - start_time
                step += 1

            except (tf.errors.OutOfRangeError, tf.errors.InvalidArgumentError) as e:
           
                print('Done training for %d epochs, %d steps.' % (num_epochs, step))
                # When done, ask the threads to stop.
                coord.request_stop()

        
        # Wait for threads to finish.
        coord.join(threads)
        sess.close()

Done training for 1 epochs, 33 steps.


In [16]:
debug_ex.ix[:,0].astype(float).reset_index(drop=True).equals((pipe_ex.sort_values(0).reset_index(drop=True).ix[:,0]))

True

In [9]:
# cost_tracker = pd.DataFrame([None,None,None,None],
#                                   index = ['Iteration','Training Cost','Training Precision', 'Validation Cost']).T
# iter_count = 0
# batch_losses = []
# batch_accs = []
# batch_debug = []
# batch_labels = []

In [17]:
with graph.as_default():
    x = tf.placeholder(tf.float32, shape=[None, 4])
    y_ = tf.placeholder(tf.float32, shape = [None, 3])
    
    W1 = tf.Variable(tf.random_normal([4,3]), name='weights', trainable=True)
    b1 = tf.Variable(tf.random_normal([3]), name = 'bias', trainable = True)
    lin_y1 = tf.matmul(x,W1) + b1
    sig_y1 = tf.sigmoid(lin_y1)
    
    W2 = tf.Variable(tf.random_normal([3,3]), name='weights', trainable=True)
    b2 = tf.Variable(tf.random_normal([3]), name = 'bias', trainable = True)
    lin_y2 = tf.matmul(sig_y1,W2) + b2
    
    smax_num = tf.transpose(tf.exp(lin_y2-tf.reduce_max(lin_y2)))
    smax_den = tf.reduce_sum(tf.exp(lin_y2-tf.reduce_max(lin_y2)),-1)
    softmax_y2 = tf.transpose(tf.divide(smax_num, smax_den))
    
    NLLCriterion = -tf.reduce_mean(tf.reduce_sum(tf.multiply(y_,tf.log(softmax_y2+1e-10)),axis=1))
    learning_rate = .1
    momentum = .2
    train_step = tf.train.MomentumOptimizer(learning_rate,momentum).minimize(NLLCriterion)
    
    unique_labels = tf.constant(["Iris-setosa", "Iris-versicolor", "Iris-virginica"], dtype=tf.string)
    
    example_feed, labels_feed = input_pipeline(train_set, possible_labels = unique_labels,
                                               batch_size = batch_size, num_epochs = num_epochs)
    v_example_feed, v_labels_feed = input_pipeline(valid_set, possible_labels = unique_labels,
                                               batch_size = valid_len, num_epochs = 1)
    
    correct = tf.equal(tf.argmax(softmax_y2,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    
    with tf.Session() as sess:
        # Start populating the filename queue.
        
        init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        sess.run(init_op)
        coord = tf.train.Coordinator()  
        threads = tf.train.start_queue_runners(coord=coord, sess=sess)
        
        step = 0
        while not coord.should_stop():
            try:
                start_time = time.time()                
               
                example_batch, label_batch = sess.run([example_feed, labels_feed])
                example_batch = example_batch[:,1:]
                
                sess.run(train_step, feed_dict = {x: example_batch, y_: label_batch})

                duration = time.time() - start_time
                step += 1

            except (tf.errors.OutOfRangeError, tf.errors.InvalidArgumentError) as e:
                
                valid_examples, valid_labels = sess.run([v_example_feed, v_labels_feed])
                valid_examples = valid_examples[:,1:]
                valid_cost = sess.run(NLLCriterion, feed_dict = {x: valid_examples, y_: valid_labels})
                acc = sess.run(accuracy, feed_dict = {x: valid_examples, y_: valid_labels})

                
                print('Done training for %d epochs, %d steps.' % (num_epochs, step))
                # When done, ask the threads to stop.
                coord.request_stop()

        
        # Wait for threads to finish.
        coord.join(threads)
        sess.close()

Done training for 1 epochs, 33 steps.


In [18]:
valid_cost

0.84150225

In [19]:
acc

0.61538464